In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2021, 12, 25), datetime.date(2021, 12, 24))

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2021, 12, 24), datetime.date(2021, 12, 17))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [3]:
sql = """DELETE FROM sales"""
rp = conlite.execute(sql)
rp.rowcount

0

In [4]:
sql = """
SELECT name
FROM stocks 
WHERE status IN ("B","I","O","S") 
ORDER BY status, name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'DOHOME', 'GLOBAL', 'NOBLE', 'PTT', 'RATCH', 'SIS', 'STA', 'BGRIM', 'CKP', 'DCC', 'DIF', 'IVL', 'KBANK', 'KCE', 'NER', 'RCL', 'RJH', 'SAT', 'TOP', 'TSTH', 'TU', 'VNT', 'ROJNA', 'SYNEX', 'TCAP'"

### Get past one year data

In [5]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2020-12-24' AND name IN ('DOHOME', 'GLOBAL', 'NOBLE', 'PTT', 'RATCH', 'SIS', 'STA', 'BGRIM', 'CKP', 'DCC', 'DIF', 'IVL', 'KBANK', 'KCE', 'NER', 'RCL', 'RJH', 'SAT', 'TOP', 'TSTH', 'TU', 'VNT', 'ROJNA', 'SYNEX', 'TCAP') 
ORDER BY name, date


In [6]:
df = pd.read_sql(sql, const)
df.shape

(6046, 7)

In [7]:
df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [8]:
sql = """
SELECT name,status 
FROM stocks 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,status
name,
BGRIM,I
CKP,I
DCC,I
DIF,I
DOHOME,B
GLOBAL,B
IVL,I
KBANK,I
KCE,I


### Create monitors from stocks

In [9]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,status
name,
BGRIM,I
CKP,I
DCC,I
DIF,I
DOHOME,B
GLOBAL,B
IVL,I
KBANK,I
KCE,I


In [10]:
monitors.shape[0]

25

In [11]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/monitors.csv',
 '\\Users\\User\\iCloudDrive\\monitors.csv',
 '\\Users\\User\\Dropbox\\monitors.csv')

In [12]:
monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [13]:
sql = """
SELECT status, COUNT(*) AS items 
FROM stocks 
WHERE name IN (%s)
GROUP BY status
ORDER BY status
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,status,items
0,B,7
1,I,15
2,O,3


In [14]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [15]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(114, 1)

In [16]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(115, 3)

In [17]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,status,_merge
20,TCAP,O,left_only


In [18]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(90, 3)

In [19]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
DOHOME,B,SET100
GLOBAL,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
SIS,B,SET
STA,B,SET50
BGRIM,I,SET50
CKP,I,SET100


In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/stocks-all.csv',
 '\\Users\\User\\iCloudDrive\\stocks-all.csv',
 '\\Users\\User\\Dropbox\\stocks-all.csv')

In [21]:
stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [22]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
DOHOME,B
GLOBAL,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O
SIS,B
STA,B
SYNEX,O


In [23]:
buy_candidates.shape[0]

10

In [24]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BGRIM,I
CKP,I
DCC,I
DIF,I
IVL,I
KBANK,I
KCE,I
NER,I
RCL,I


In [25]:
sell_candidates.shape[0]

15